In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
%run read_data.ipynb

['kiz', 'kork', 'mutlu', 'notr', 'uzul']
['kiz.txt', 'kork.txt', 'mutlu.txt', 'notr.txt', 'uzul.txt']
3316
3316


In [10]:
all_text[:10]

['çok kız ne kadar ayıp şey',
 'koku çok kız çünkü ayva ben tek arkadaş  insan var kendi git gibi tek arkadaş götür yan',
 'hayır çok sev dizi yani aşırı kız',
 'buna kız de evet ona kız ney kız kızneg sen mantık göre belirleneg kusur bakneg',
 'kız ve çok üzül',
 'kız bu geç ama aynı zaman kır bu nasıl geç',
 'of ben yet çok kız karakter  ne güncelneg yap herkes vere zor ne adam',
 'bak kendi çok kız şu sen muhatap al için oku için bekle',
 'sonra durul küs kız biz bu kadar gönül bu takım sar o herkes met et futbol defans ton',
 'evet çok sev hem de gül bir durum görneg sine çok kız']

In [11]:
# Create train - test splits
# test data will never be seen by the classifiers. 
# This left-out data will be used to test the NN as well.
import random
import numpy as np

# parameters #
train_ratio= 0.8
test_ratio= 0.2
SEED= 1
random.seed= SEED
#########

all_data_count = len(all_text)
indices = list(range(all_data_count))
np.random.shuffle(indices) # first, shuffle the data! Do not miss this part!

split = int(np.floor(test_ratio * all_data_count))
train_idx, test_idx = indices[split:], indices[:split]


In [12]:
# test data will never be seen by the classifiers. This left-out data will be used to test the NN as well.

all_text_train= [all_text[i] for i in train_idx]
labels_train= [labels[i] for i in train_idx]
all_text_test= [all_text[i] for i in test_idx]
labels_train=  [labels[i] for i in test_idx]

In [13]:
all_text_train[0]

'kimi gönül alla ile mutlu ol onu gör herkes göz aydın ol kahya muz'

In [14]:
# pre-process training data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

NameError: name 'helper' is not defined

In [18]:
# create words counts dictionary and the vocabulary:
total_counts={}
for i in range(len(all_text_train)):
    for word in all_text_train[i].split(" "):
        total_counts[word] = total_counts.get(word,0) +1

vocab = set(total_counts.keys())
vocab_size = len(vocab)
print(vocab_size)

3323


In [19]:
layer_0 = np.zeros((1,vocab_size))

In [ ]:
# Create a dictionary of words in the vocabulary mapped to index positions 
# (to be used in layer_0)
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
    
# display the map of words to indices
word2index

In [42]:
def update_input_layer(review):
    """ Modify the global layer_0 to represent the vector form of review.
    The element at a given index of layer_0 should represent
    how many times the given word occurs in the review.
    Args:
        review(string) - the string of the review
    Returns:
        None
    """
     
    global layer_0
    
    # clear out previous state, reset the layer to be all 0s
    layer_0 *= 0
    
    # count how many times each word is used in the given review and store the results in layer_0 
    for word in review.split(" "):
        layer_0[0][word2index[word]] += 1

In [43]:
update_input_layer(all_text_train[0])

In [48]:
def get_target_for_label(label):
    if(label == 'kiz'):
        return 1
    elif (label == 'kork'):
        return 2
    elif (label == 'mutlu'):
        return 3
    elif (label == 'uzul'):
        return 4
    else:
        return 0 # neutral

In [49]:
get_target_for_label(labels_train[0])

1

In [50]:
all_text_train[0]

'of alt kendi adım gör hayat küs'

## Building a Neural Network
- Do **not** add a non-linearity in the hidden layer. That is, do not use an activation function when calculating the hidden layer outputs.

In [8]:
# WITH UNIGRAM COUNT VECTORIZER:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# The folds are made by preserving the percentage of samples for each class.
# use the original data, all_text!

list_classifier= [MultinomialNB(), LinearSVC()]

for clf in list_classifier:
    print(clf)
    skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=123)
    all_text= np.array(all_text)
    labels= np.array(labels)
    scores= []
    i= 1
    for train_index, test_index in skf.split(all_text, labels):
        X_train, y_train = all_text[train_index], labels[train_index] 
        X_test, y_test = all_text[test_index], labels[test_index]

        train_vectorizer = CountVectorizer()
        X_train = train_vectorizer.fit_transform(X_train)
        train_vocab= train_vectorizer.vocabulary_   
        test_vectorizer = CountVectorizer(vocabulary=train_vocab)
        X_test = test_vectorizer.fit_transform(X_test)
        clf.fit(X_train, y_train)
        sc= accuracy_score(y_test, clf.predict(X_test))
        scores.append(sc)
        print("fold "+ str(i)+ " accuracy: "+ str(sc))
        i+= 1
    print("Mean score: "+ str(np.mean(scores)))
    print(" ")

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
fold 1 accuracy: 0.8345864661654135
fold 2 accuracy: 0.8463855421686747
fold 3 accuracy: 0.8491704374057315
fold 4 accuracy: 0.8235294117647058
fold 5 accuracy: 0.8350983358547656
Mean score: 0.8377540386718583
 
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
fold 1 accuracy: 0.8661654135338346
fold 2 accuracy: 0.8780120481927711
fold 3 accuracy: 0.8778280542986425
fold 4 accuracy: 0.8778280542986425
fold 5 accuracy: 0.8774583963691377
Mean score: 0.8754583933386056
 


In [9]:
# WITH UNIGRAM TF VECTORIZER:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# The folds are made by preserving the percentage of samples for each class.
# use the original data, all_text!

from sklearn.feature_extraction.text import TfidfTransformer

list_classifier= [MultinomialNB(), LinearSVC()]

for clf in list_classifier:
    print(clf)
    skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=123)
    all_text= np.array(all_text)
    labels= np.array(labels)
    scores= []
    i= 1
    for train_index, test_index in skf.split(all_text, labels):
        X_train, y_train = all_text[train_index], labels[train_index] 
        X_test, y_test = all_text[test_index], labels[test_index]

        train_vectorizer = CountVectorizer()
        X_train = train_vectorizer.fit_transform(X_train)
        transformer_tf= TfidfTransformer(use_idf= False, norm= 'l2') # if normalize None, it is the same as CountVect
        # l2 normalization: like percentage of values
        X_train= transformer_tf.fit_transform(X_train)
           
        train_vocab= train_vectorizer.vocabulary_   
        test_vectorizer = CountVectorizer(vocabulary=train_vocab)
        X_test = test_vectorizer.fit_transform(X_test)
        X_test= transformer_tf.fit_transform(X_test)
        
        clf.fit(X_train, y_train)
        sc= accuracy_score(y_test, clf.predict(X_test))
        scores.append(sc)
        print("fold "+ str(i)+ " accuracy: "+ str(sc))
        i+= 1
    print("Mean score: "+ str(np.mean(scores)))
    print(" ")



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
fold 1 accuracy: 0.8
fold 2 accuracy: 0.7966867469879518
fold 3 accuracy: 0.8054298642533937
fold 4 accuracy: 0.77526395173454
fold 5 accuracy: 0.81089258698941
Mean score: 0.7976546299930591
 
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
fold 1 accuracy: 0.8857142857142857
fold 2 accuracy: 0.8855421686746988
fold 3 accuracy: 0.8868778280542986
fold 4 accuracy: 0.8883861236802413
fold 5 accuracy: 0.8835098335854765
Mean score: 0.8860060479418002
 


In [10]:
# WITH UNIGRAM TF-idf VECTORIZER:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import StratifiedKFold 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# The folds are made by preserving the percentage of samples for each class.
# use the original data, all_text!


from sklearn.feature_extraction.text import TfidfTransformer

list_classifier= [MultinomialNB(), LinearSVC()]

for clf in list_classifier:
    print(clf)
    skf = StratifiedKFold(n_splits=5,shuffle=True, random_state=123)
    all_text= np.array(all_text)
    labels= np.array(labels)
    scores= []
    i= 1
    for train_index, test_index in skf.split(all_text, labels):
        X_train, y_train = all_text[train_index], labels[train_index] 
        X_test, y_test = all_text[test_index], labels[test_index]

        train_vectorizer = CountVectorizer()
        X_train = train_vectorizer.fit_transform(X_train)
        transformer_tf= TfidfTransformer(use_idf= True, norm= 'l2', smooth_idf=True) 
        # if normalize None, it is the same as CountVect
        # l2 normalization: like percentage of values
        X_train= transformer_tf.fit_transform(X_train)
           
        train_vocab= train_vectorizer.vocabulary_   
        test_vectorizer = CountVectorizer(vocabulary=train_vocab)
        X_test = test_vectorizer.fit_transform(X_test)
        X_test= transformer_tf.fit_transform(X_test)
        
        clf.fit(X_train, y_train)
        sc= accuracy_score(y_test, clf.predict(X_test))
        scores.append(sc)
        print("fold "+ str(i)+ " accuracy: "+ str(sc))
        i+= 1
    print("Mean score: "+ str(np.mean(scores)))
    print(" ")



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
fold 1 accuracy: 0.8
fold 2 accuracy: 0.8012048192771084
fold 3 accuracy: 0.8069381598793364
fold 4 accuracy: 0.7782805429864253
fold 5 accuracy: 0.8093797276853253
Mean score: 0.799160649965639
 
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
fold 1 accuracy: 0.8872180451127819
fold 2 accuracy: 0.8855421686746988
fold 3 accuracy: 0.8838612368024132
fold 4 accuracy: 0.8838612368024132
fold 5 accuracy: 0.8789712556732224
Mean score: 0.8838907886131059
 
